In [ ]:
%%capture
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alinatl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM
import numpy as np
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from typing import Union
from transformers import EvalPrediction

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = NllbTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

tokenizer.src_lang = 'eng_Latn'
tokenizer.tgt_lang = 'kir_Cyrl'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Load data

In [ ]:
import os
from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
from datasets import load_dataset

kir = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="https://huggingface.co/datasets/openlanguagedata/oldi_seed/resolve/refs/pr/4/seed/kir_Cyrl.parquet"
)['train']

eng = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="seed/eng_Latn.parquet"
)["train"]

eng_dict = {row["id"]: row["text"] for row in eng}
merged = kir.map(lambda row: {"kir": row["text"], "eng": eng_dict.get(row["id"], None)})
train_dataset = merged.select_columns(["eng", "kir"])

print(train_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 6193
})

In [ ]:
eng_ds = load_dataset("openlanguagedata/flores_plus", "eng_Latn", split="dev")
kir_ds = load_dataset("openlanguagedata/flores_plus", "kir_Cyrl", split="dev")

assert len(eng_ds) == len(kir_ds)

parallel_data = [
    {"eng": eng_ds[i]["text"], "kir": kir_ds[i]["text"]}
    for i in range(len(eng_ds))
]

test_dataset = Dataset.from_list(parallel_data)
test_dataset

README.md:   0%|          | 0.00/72.5k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/220 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/214 [00:00<?, ?it/s]

dev/eng_Latn.parquet:   0%|          | 0.00/112k [00:00<?, ?B/s]

devtest/eng_Latn.parquet:   0%|          | 0.00/117k [00:00<?, ?B/s]

...

dev/kir_Cyrl.parquet:   0%|          | 0.00/150k [00:00<?, ?B/s]

devtest/kir_Cyrl.parquet:   0%|          | 0.00/157k [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 997
})

## Тестовый код

In [ ]:
x = tokenizer(['My name is Alina'], return_tensors='pt', padding=True, truncation=True, max_length=64)
print(x)

y = tokenizer(['Менин атым Алина.', ], return_tensors='pt', padding=True, truncation=True, max_length=64)
y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100
print(y)

{'input_ids': tensor([[256047,   7177,  17606,    248,    671,    231,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[256047, 108332,    218,  28923,  30694,    106, 248075,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
translated_tokens = model.generate(
    **y, forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"), max_length=30
)
translated_tokens

tensor([[     2, 256047,   7177,  17606,    248,    671,    231, 248075,      2]])

In [ ]:
batch = tokenizer(['My name is Alina.'], return_tensors='pt', padding=True, truncation=True, max_length=64)
print(batch)
print(tokenizer.convert_ids_to_tokens(batch['input_ids'][0]))
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']
labels = tokenizer(['Менин атым Алина.'], return_tensors='pt', padding=True, truncation=True, max_length=64)['input_ids']
labels[labels == tokenizer.pad_token_id] = -100


outputs = model(input_ids=input_ids,attention_mask=attention_mask, labels=labels)
outputs_2 = model.generate(input_ids=input_ids, forced_bos_token_id=tokenizer.convert_tokens_to_ids("kir_Cyrl"), max_length=30)
print(outputs_2[0])
print(tokenizer.convert_ids_to_tokens(outputs_2[0]))
print(tokenizer.batch_decode(outputs_2, skip_special_tokens=True))
outputs.loss

{'input_ids': tensor([[256047,   7177,  17606,    248,    671,    231, 248075,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
['eng_Latn', '▁My', '▁name', '▁is', '▁Al', 'ina', '.', '</s>']


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


tensor([     2, 256095, 108332,    218,  28923,  30694,    106, 248075,      2])
['</s>', 'kir_Cyrl', '▁Менин', '▁а', 'тым', '▁Али', 'на', '.', '</s>']
['Менин атым Алина.']


tensor(3.8798, grad_fn=<NllLossBackward0>)

## dataloader

In [ ]:
from torch.utils.data import DataLoader, Dataset

def tokenize_data(dataset, source_col='eng', target_col='kir', tokenizer=None, max_length=128):
    tokenizer.src_lang = 'eng_Latn'
    source_tokenized = tokenizer(
        list(dataset[source_col]),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length
    )
    tokenizer.src_lang = 'kir_Cyrl'
    target_tokenized = tokenizer(
        list(dataset[target_col]),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length
    )
    return source_tokenized, target_tokenized


class TranslationDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }
max_length = 128
source_tokenized_train, target_tokenized_train = tokenize_data(train_dataset, tokenizer=tokenizer, max_length=max_length)
source_tokenized_test, target_tokenized_test = tokenize_data(test_dataset, tokenizer=tokenizer, max_length=max_length)

input_ids_train = source_tokenized_train['input_ids']
attention_mask_train = source_tokenized_train['attention_mask']
labels_train = target_tokenized_train['input_ids'].clone()
labels_train[labels_train == tokenizer.pad_token_id] = -100

input_ids_test = source_tokenized_test['input_ids']
attention_mask_test = source_tokenized_test['attention_mask']
labels_test = target_tokenized_test['input_ids'].clone()
labels_test[labels_test == tokenizer.pad_token_id] = -100

print("Train input_ids shape:", input_ids_train.shape)
print("Train attention_mask shape:", attention_mask_train.shape)
print("Train labels shape:", labels_train.shape)

print("Test input_ids shape:", input_ids_test.shape)
print("Test attention_mask shape:", attention_mask_test.shape)
print("Test labels shape:", labels_test.shape)

train_dataset = TranslationDataset(input_ids_train, attention_mask_train, labels_train)
test_dataset = TranslationDataset(input_ids_test, attention_mask_test, labels_test)

Train input_ids shape: torch.Size([6193, 121])
Train attention_mask shape: torch.Size([6193, 121])
Train labels shape: torch.Size([6193, 128])
Test input_ids shape: torch.Size([997, 80])
Test attention_mask shape: torch.Size([997, 80])
Test labels shape: torch.Size([997, 85])


In [ ]:
print(tokenizer.convert_ids_to_tokens(test_dataset[0]['input_ids']))

['eng_Latn', '▁On', '▁Monday', ',', '▁scientists', '▁from', '▁the', '▁Stanford', '▁University', '▁School', '▁of', '▁Medicine', '▁announced', '▁the', '▁inv', 'ention', '▁of', '▁a', '▁new', '▁diagnostic', '▁tool', '▁that', '▁can', '▁sort', '▁cells', '▁by', '▁type', ':', '▁a', '▁tiny', '▁pr', 'inta', 'ble', '▁chip', '▁that', '▁can', '▁be', '▁manufact', 'ured', '▁using', '▁standard', '▁ink', 'jet', '▁prin', 'ters', '▁for', '▁possi', 'bly', '▁about', '▁one', '▁U', '.', 'S', '.', '▁cent', '▁each', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
import gc
import torch

def cleanup():
    """ A function to clean memory (mostly to prevent GPU memory leaks) """
    gc.collect()
    torch.cuda.empty_cache()
cleanup()

## Training

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
chrfpp = evaluate.load("chrf")

In [ ]:
def compute_metrics(eval_pred: Union[EvalPrediction, tuple], compute_result=False):

    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    if predictions.ndim == 3:
        predictions = np.argmax(predictions, axis=-1)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_predictions = [pred.strip() for pred in decoded_predictions]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu.add_batch(predictions=decoded_predictions, references=decoded_labels)
    chrfpp.add_batch(predictions=decoded_predictions, references=decoded_labels)

    if compute_result:
        bleu_result = bleu.compute()
        chrfpp_result = chrfpp.compute()

        print(f"CHRF++: {chrfpp_result['score']:.4f} | BLEU: {bleu_result['bleu']:.4f}")

        return {
            "chrf++": chrfpp_result['score'],
            "bleu": bleu_result["bleu"],
        }

    return {}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"/content/nllb-en-ky",
    num_train_epochs=4,
    eval_strategy='steps',
    logging_steps=90,
    eval_steps = 90,
    torch_empty_cache_steps=90,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=4,
    gradient_accumulation_steps=8,
    warmup_steps=10,
    weight_decay=1e-2,
    optim="adamw_torch_fused",
    save_total_limit=2,
    learning_rate=5e-5,
    batch_eval_metrics=True,
    report_to="wandb",
    save_strategy="no",
    push_to_hub=True,
    hub_model_id="alinatl/nllb-en-ky",
    fp16=True,
)

In [ ]:
train_size = len(train_dataset)
batch_size_effective = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
steps_per_epoch = train_size / batch_size_effective

logging_frequency_epochs = training_args.logging_steps / steps_per_epoch

print(f"Шагов в эпохе: {steps_per_epoch}")
print(f"Логгинг каждые {training_args.logging_steps} шагов = {logging_frequency_epochs:.2f} эпохи")


Шагов в эпохе: 193.53125
Логгинг каждые 90 шагов = 0.47 эпохи


In [ ]:
len(train_dataset) / (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * 1

193.53125

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    forced_bos_token_id=tokenizer.convert_tokens_to_ids("kir_Cyrl"),
    max_length=max_length,
    num_beams=1,
)

model.generation_config = generation_config

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(project='NMT-en-ky')
trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss,Chrf++,Bleu
90,1.199300,1.574046,53.086395,0.057913
180,1.067900,1.570270,53.235500,0.058485
270,0.960500,1.594411,53.669395,0.065470
360,0.916200,1.595004,53.955998,0.067718
450,0.859200,1.610906,53.933869,0.067216
540,0.829300,1.623512,54.091164,0.072395


CHRF++: 53.0864 | BLEU: 0.0579
CHRF++: 53.2355 | BLEU: 0.0585
CHRF++: 53.6694 | BLEU: 0.0655
CHRF++: 53.9560 | BLEU: 0.0677
CHRF++: 53.9339 | BLEU: 0.0672
CHRF++: 54.0912 | BLEU: 0.0724


KeyboardInterrupt: 

In [ ]:
from huggingface_hub import create_repo
create_repo("alinatl/nllb-en-ky", private=False)

RepoUrl('https://huggingface.co/alinatl/nllb-en-ky', endpoint='https://huggingface.co', repo_type='model', repo_id='alinatl/nllb-en-ky')

In [ ]:
trainer.push_to_hub('alinatl/nllb-en-ky')

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  .../nllb-en-ky/sentencepiece.bpe.model: 100%|##########| 4.85MB / 4.85MB            

  /content/nllb-en-ky/model.safetensors :   0%|          | 27.0kB / 2.46GB            

  /content/nllb-en-ky/training_args.bin :   8%|8         |   492B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/alinatl/nllb-en-ky/commit/c65815fb69435de086260a68fac8549ed62fb217', commit_message='alinatl/nllb-en-ky', commit_description='', oid='c65815fb69435de086260a68fac8549ed62fb217', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alinatl/nllb-en-ky', endpoint='https://huggingface.co', repo_type='model', repo_id='alinatl/nllb-en-ky'), pr_revision=None, pr_num=None)